In [1]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM
from torch.nn.functional import log_softmax
import torch
import numpy as np
import json
from matplotlib import pyplot as plt

/Users/aron/codebase/uniwork/cicl/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [3]:
def encode(string):
    input_ids = tokenizer.encode(string, return_tensors='pt', add_special_tokens=False)
    input_ids = torch.cat([torch.tensor([[tokenizer.bos_token_id]]), input_ids, torch.tensor([[tokenizer.eos_token_id]])], dim=1)
    return input_ids

In [4]:
def perplexity(out, labels):
    sm = log_softmax(out.logits, dim=-1)
    probs = []
    for i in range(labels.shape[1] - 1):
        next_idx = labels[0, i + 1].item()
        scores = sm[0, i, :]
        prob_next_idx = scores[next_idx].item()
        probs.append(prob_next_idx)

    probs = np.array(probs)
    probs = np.sum(probs)
    l = labels.shape[1] - 1
    ppl = - (1 / l) * np.sum(probs)
    return np.exp(ppl)

def perplexity_seq(seq):
    input_ids = encode(seq)
    out = model(input_ids)
    return perplexity(out, input_ids)

perplexity_seq("I enjoy walking with my cute dog", tokenizer=tokenizer, model=model)

208.3806056168213

In [5]:
taskC_train = []
with open ("src/data/subtaskC_train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if line:
            parsed = json.loads(line)
            taskC_train.append((parsed["text"], parsed["label"], parsed["id"]))
        

In [8]:
logits = []
window_size = 1024
slide_amount = 256

for i, (text, label, id) in enumerate(taskC_train):
    input_ids = encode(text)
    if input_ids.shape[1] > window_size:
        continue
    

3649